# ETL 2. Transformación 1 - Limpieza

In [118]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast 

from datetime import datetime, timedelta

pd.options.display.max_columns=None
pd.options.display.max_rows=None

## Introducción:
Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

**El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes**. 

Para ello:
1. Cargaremos los dos ficheros de datos del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea  
  

In [119]:
clima = pd.read_pickle('datos/ETL-1.pkl')
clima.head()


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA
2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",12,15,2,1028,none,0,0,310,2,USA
3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 270, 'speed':...",14,15,1,1028,none,0,0,60,2,USA
4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",14,15,3,1027,none,0,0,85,2,USA


In [120]:
df = pd.read_csv('datos/attacks_limpieza_completa.csv', index_col=0)
df.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.658512,-102.007710,HN,1,0,0,1,0,0,2,7,4,6.313925


 
2. Del dataframe de los datos climáticos seleccionaremos todas las columnas.
Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. 

A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. 

⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.


In [121]:
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.


# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes


timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
temp2m                int64
lifted_index          int64
rh2m                  int64
msl_pressure          int64
prec_type            object
prec_amount           int64
snow_depth            int64
wind10m.direction    object
wind10m.speed         int64
country              object
dtype: object

````python

#en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información.

In [122]:
list_country = ['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

In [123]:
df_country = df[df['country'].isin(list_country)]
df_country.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925


In [124]:
df_country['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [125]:
clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA


In [126]:
df_rh = clima['rh_profile'].apply(pd.Series)
df_rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': -2}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 4}"
1,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"


In [127]:
df_wind = clima['wind_profile'].apply(pd.Series)
df_wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 210, 'speed': 2}","{'layer': '900mb', 'direction': 250, 'speed': 2}","{'layer': '850mb', 'direction': 300, 'speed': 3}","{'layer': '800mb', 'direction': 305, 'speed': 4}","{'layer': '750mb', 'direction': 315, 'speed': 4}","{'layer': '700mb', 'direction': 320, 'speed': 5}","{'layer': '650mb', 'direction': 315, 'speed': 5}","{'layer': '600mb', 'direction': 310, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 305, 'speed': 6}","{'layer': '450mb', 'direction': 310, 'speed': 6}","{'layer': '400mb', 'direction': 325, 'speed': 6}","{'layer': '350mb', 'direction': 330, 'speed': 7}","{'layer': '300mb', 'direction': 335, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 8}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
1,"{'layer': '950mb', 'direction': 345, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 355, 'speed': 3}","{'layer': '800mb', 'direction': 325, 'speed': 3}","{'layer': '750mb', 'direction': 315, 'speed': 3}","{'layer': '700mb', 'direction': 310, 'speed': 4}","{'layer': '650mb', 'direction': 310, 'speed': 5}","{'layer': '600mb', 'direction': 305, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 310, 'speed': 5}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 330, 'speed': 7}","{'layer': '350mb', 'direction': 325, 'speed': 7}","{'layer': '300mb', 'direction': 325, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 7}","{'layer': '200mb', 'direction': 340, 'speed': 8}"


In [128]:
x = df_rh[0].apply(pd.Series)
x.head(2)

,layer,rh
0,950mb,3
1,950mb,2


In [133]:
for i in range(len(df_rh.columns)): 
    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_rh[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

ValueError: Length of values (320) does not match length of index (6218)

In [129]:
df_limpio = pd.DataFrame()


for col in range(len(df_rh.columns)): 
    df_rh_limp = df_rh[col].apply(pd.Series)

    nombre = "rh_" + df_rh_limp['layer'][0]
    valor = df_rh_limp['rh'][1]
# rh y layer son keys, sin ellas no podemos usar los índices.




0    rh_200mb
0    rh_200mb
0    rh_200mb
0    rh_200mb
0    rh_200mb
Name: layer, dtype: object 1     7
1    10
1     6
1    15
1    -1
Name: rh, dtype: int64


In [131]:
df_limpio.head()

""



4. Guardad los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.